# Synthetic Optical Flow from Fused Lidar


In [1]:
import sys
sys.path.append('/opt/psegs')

import numpy as np

from psegs.exp.fused_lidar import FusedLidarCloudTableBase

import IPython.display
import PIL.Image


## General Notebook Utilities
    
def imshow(x):
    IPython.display.display(PIL.Image.fromarray(x))

def show_html(x):
    from IPython.core.display import display, HTML
    display(HTML(x))

## SemanticKITTI

In [2]:
from psegs.exp.semantic_kitti import SemanticKITTISDTable
class SemanticKITTIFusedWorldCloudTable(FusedLidarCloudTableBase):
    SRC_SD_TABLE = SemanticKITTISDTable

    # SemanticKITTI has no cuboids, so we skip this step.
    HAS_OBJ_CLOUDS = False
    
    @classmethod
    def _get_task_lidar_cuboid_rdd(cls, spark, segment_uri):
        seg_rdd = cls.SRC_SD_TABLE.get_segment_datum_rdd(spark, segment_uri)
        
        # SemanticKITTI has no cuboids, so the Fuser algo simply concats the cloud points
        def iter_task_rows(iter_sds):
            from pyspark import Row
            from oarphpy.spark import RowAdapter
            for sd in iter_sds:
                if sd.point_cloud is not None:
                    pc = sd.point_cloud
                    task_id = "%s.%s" % (sd.uri.segment_id, pc.extra['semantic_kitti.scan_id'])
                    yield Row(
                        task_id=task_id,
                        point_clouds=[pc],
                        cuboids=[])
        
        task_rdd = seg_rdd.mapPartitions(iter_task_rows)
        import pyspark
        task_rdd = task_rdd.persist(pyspark.StorageLevel.DISK_ONLY)
        return task_rdd
        


## KITTI-360

In [3]:
from psegs.datasets.kitti_360 import KITTI360SDTable
class KITTI360OurFusedClouds(KITTI360SDTable):
    INCLUDE_FISHEYES = False
    INCLUDE_FUSED_CLOUDS = False  # Use our own fused clouds

class KITTI360OurFusedWorldCloudTable(FusedLidarCloudTableBase):
    SRC_SD_TABLE = KITTI360OurFusedClouds
    
    @classmethod
    def _get_task_lidar_cuboid_rdd(cls, spark, segment_uri):
        datum_df = cls.SRC_SD_TABLE.get_segment_datum_df(spark, segment_uri)
        datum_df.registerTempTable('datums')
        spark.catalog.dropTempView('culi_tasks_df')
        print('Building tasks table for %s ...' % segment_uri.segment_id)
        spark.sql("""
          CACHE TABLE culi_tasks_df OPTIONS ( 'storageLevel' 'DISK_ONLY' ) AS
          SELECT 
              CONCAT(uri.segment_id, '.', uri.extra.`kitti-360.frame_id`) AS task_id,
              FLATTEN(COLLECT_LIST(cuboids)) AS cuboids, 
              COLLECT_LIST(point_cloud) AS point_clouds
          FROM datums
          WHERE 
              uri.topic LIKE '%cuboid%' OR uri.topic LIKE '%lidar%'
          GROUP BY task_id
          LIMIT 100
        """)
        tasks_df = spark.sql('SELECT * FROM culi_tasks_df')
        print('... done.')
        return tasks_df.rdd


## NuScenes

In [4]:
# !pip3 install nuscenes-devkit==1.1.2
from psegs.datasets.nuscenes import NuscStampedDatumTableBase, NuscStampedDatumTableLabelsAllFrames
class NuscFusedWorldCloudTable(FusedLidarCloudTableBase):
    SRC_SD_TABLE = NuscStampedDatumTableBase
    
    SPLITS = ['train_detect', 'train_track']
    
    @classmethod
    def _filter_ego_vehicle(cls, cloud_ego):
        cloud_ego = cloud_ego[np.where(  ~(
                        (cloud_ego[:, 0] <= 1.5) & (cloud_ego[:, 0] >= -1.5) &  # Nusc lidar +x is +right
                        (cloud_ego[:, 1] <= 2.5) & (cloud_ego[:, 0] >= -2.5) &  # Nusc lidar +y is +forward
                        (cloud_ego[:, 1] <= 1.5) & (cloud_ego[:, 0] >= -1.5)   # Nusc lidar +z is +up
        ))]
        return cloud_ego
    
    @classmethod
    def _get_task_lidar_cuboid_rdd(cls, spark, segment_uri):
        datum_df = cls.SRC_SD_TABLE.get_segment_datum_df(spark, segment_uri)
        datum_df.registerTempTable('datums')
        spark.catalog.dropTempView('culi_tasks_df')
        print('Building tasks table for %s ...' % segment_uri.segment_id)
        if cls.SRC_SD_TABLE.LABELS_KEYFRAMES_ONLY:
            # For Nusc: group by nuscenes-sample-token WITH KEYFRAMES
            spark.sql("""
              CACHE TABLE culi_tasks_df OPTIONS ( 'storageLevel' 'DISK_ONLY' ) AS
              SELECT 
                  uri.extra.`nuscenes-sample-token` AS task_id,
                  FLATTEN(COLLECT_LIST(cuboids)) AS cuboids, 
                  COLLECT_LIST(point_cloud) AS point_clouds
              FROM datums
              WHERE 
                uri.extra.`nuscenes-is-keyframe` = 'True' AND (
                  uri.extra['nuscenes-label-channel'] is NULL OR 
                  uri.extra['nuscenes-label-channel'] LIKE '%LIDAR%'
                ) AND (
                  uri.topic LIKE '%cuboid%' OR
                  uri.topic LIKE '%lidar%'
                )
              GROUP BY task_id
            """)
        else:
            # For Nusc: group by nuscenes-sample-token WITH ALL FRAMES
            spark.sql("""
              CACHE TABLE culi_tasks_df OPTIONS ( 'storageLevel' 'DISK_ONLY' ) AS
              SELECT 
                  CONCAT(uri.segment_id, '.', uri.timestamp) AS task_id,
                  FLATTEN(COLLECT_LIST(cuboids)) AS cuboids, 
                  COLLECT_LIST(point_cloud) AS point_clouds
              FROM datums
              WHERE 
                (
                  uri.extra['nuscenes-label-channel'] is NULL OR 
                  uri.extra['nuscenes-label-channel'] LIKE '%LIDAR%'
                ) AND (
                  uri.topic LIKE '%cuboid%' OR
                  uri.topic LIKE '%lidar%'
                )
              GROUP BY task_id
              HAVING SIZE(cuboids) > 0 AND SIZE(point_clouds) > 0
            """)
        
        tasks_df = spark.sql('SELECT * FROM culi_tasks_df')
        print('... done.')
        return tasks_df.rdd


## Start Spark

In [5]:
from psegs.spark import NBSpark
spark = NBSpark.getOrCreate()

2021-02-13 21:38:42,885	oarph 726314 : Using source root /opt/psegs/psegs 
2021-02-13 21:38:42,886	oarph 726314 : Using source root /opt/psegs 
2021-02-13 21:38:42,930	oarph 726314 : Generating egg to /tmp/tmpr_ovivrq_oarphpy_eggbuild ...
2021-02-13 21:38:43,010	oarph 726314 : ... done.  Egg at /tmp/tmpr_ovivrq_oarphpy_eggbuild/psegs-0.0.0-py3.8.egg


## Build Fused Lidar Assets

```
docker --context default run -it --name=potree_viewer --rm --net=host -v `pwd`:/shared  jonazpiazu/potree
```

In [ ]:
T = NuscFusedWorldCloudTable
seg_uris = T.get_all_segment_uris()
samp = T.get_sample(seg_uris[10], spark=spark)


2021-02-13 21:38:46,162	ps   726314 : Filtering to only 1 segments
2021-02-13 21:38:46,163	ps   726314 : NuscFusedWorldCloudTable building fused clouds ...
2021-02-13 21:38:46,164	ps   726314 : ... have 1 segments to fuse ...
2021-02-13 21:38:46,164	ps   726314 : ... working on scene-0464 ...
2021-02-13 21:38:47,229	ps   726314 : Filtering to only 1 segments


Building tasks table for scene-0464 ...


2021-02-13 21:39:06,065	ps   726314 : Building world cloud to /opt/psegs/dataroot/fused_world_clouds/naive_cuboid_scrubber/nuscenes-Lkfo+lseg/train_detect/scene-0464/fused_world.ply ...


... done.


2021-02-13 21:39:06,977	ps   726314 : ... have 41 fusion tasks ...
2021-02-13 21:41:04,179	ps   726314 : ... computed world cloud for scene-0464 of shape (796491, 3) (0.02 GB) ...
2021-02-13 21:41:04,180	ps   726314 : ... writing ply to /opt/psegs/dataroot/fused_world_clouds/naive_cuboid_scrubber/nuscenes-Lkfo+lseg/train_detect/scene-0464/fused_world.ply ...
2021-02-13 21:41:05,068	ps   726314 : ... done writing ply.
INFO - 2021-02-13 21:41:05,068 - fused_lidar - ... done writing ply.
2021-02-13 21:41:05,075	oarph 726314 : 
ComputeWorldClouds [Pid:726314 Id:140426301035712]
-----------------------  --------------------------------------------------------------------
Thruput
N thru                   41 (of 41)
N chunks                 42
Total time               1 minute and 58.07 seconds
Total thru               19.12 MB
Rate                     161.9 KB / sec
Hz                       0
Progress
Percent Complete         100.000000
Est. Time To Completion  0 seconds
Latency (per chunk)


In [ ]:
print([lc.sensor_name for lc in samp.lidar_clouds][:10])
c = samp.lidar_clouds[0]#[lc for lc in samp.lidar_clouds if lc.sensor_name == '11002'][0]
print(c.get_cloud().shape)
imshow(c.get_bev_debug_image())
imshow(c.get_front_rv_debug_image())